# 🌍 ST-CDGM Training & Evaluation Notebook
## Spatio-Temporal Causal Diffusion Generative Model

Ce notebook vous permet de :
1. **Explorer les données** climatiques NetCDF
2. **Entraîner le modèle** ST-CDGM
3. **Tester et évaluer** les performances
4. **Visualiser les résultats** de downscaling


## 🎯 Motivation et Idée Centrale

### 💡 **Motivation**
Remplacer le bruit statistique aléatoire par un **signal physique** généré causalement

### 🔬 **Idée**
Utiliser la causalité pour générer les **détails fins** (incertitudes)

---

**Concept clé du ST-CDGM** : Au lieu d'utiliser du bruit gaussien purement aléatoire comme les modèles de diffusion classiques, le ST-CDGM génère les détails haute résolution en exploitant les relations causales entre les variables climatiques. Cela permet de produire des prédictions physiquement cohérentes et réalistes.


## 📦 1. Installation et Imports


In [ ]:
# Installation des dépendances (décommenter si nécessaire)
# !pip install torch xarray pandas numpy matplotlib seaborn xbatcher netcdf4 h5netcdf hydra-core


In [ ]:
# Configuration complète pour exécution locale
# Auto-reload des modules locaux (utile si vous modifiez le code dans src/ pendant la session)
%load_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Définir le répertoire de travail
project_root = Path.cwd()
os.chdir(project_root)

# Ajouter src au PYTHONPATH
if str(project_root / "src") not in sys.path:
    sys.path.insert(0, str(project_root / "src"))

# Imports scientifiques
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors  # Pour CenteredNorm (centrage de colormap)
import seaborn as sns

# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# Imports depuis le package st_cdgm
from st_cdgm import (
    NetCDFDataPipeline,
    HeteroGraphBuilder,
    IntelligibleVariableEncoder,
    RCNCell,
    RCNSequenceRunner,
    CausalDiffusionDecoder,
    train_epoch,
)

# Pour IntelligibleVariableConfig (si nécessaire)
from st_cdgm.models.intelligible_encoder import IntelligibleVariableConfig

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Vérifications
print(f"✅ Répertoire de travail: {os.getcwd()}")
print(f"✅ PYTHONPATH configuré: {str(project_root / 'src') in sys.path}")
print(f"✅ Package st_cdgm disponible: {Path('src/st_cdgm').exists()}")
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")
print(f"✅ Tous les imports réussis")


## 🗂️ 2. Exploration des Données

### 2.1 Configuration des chemins


In [ ]:
# Chemins relatifs depuis le répertoire du projet
# Structure data/ : raw/train/ (entraînement), raw/normalization_coefs/, raw/static_predictors/, raw/*.nc (test/eval)
from pathlib import Path
DATA_ROOT = Path("data/raw")
LR_PATH = str(DATA_ROOT / "train" / "predictor_ACCESS-CM2_hist.nc")
HR_PATH = str(DATA_ROOT / "train" / "pr_ACCESS-CM2_hist.nc")
STATIC_PATH = str(DATA_ROOT / "static_predictors" / "ERA5_eval_ccam_12km.198110_NZ_Invariant.nc")
MEAN_PATH = str(DATA_ROOT / "normalization_coefs" / "mean_1974_2011.nc")
STD_PATH = str(DATA_ROOT / "normalization_coefs" / "std_1974_2011.nc")

# Vérifier l'existence des fichiers
from pathlib import Path
for path, name in [(LR_PATH, "LR"), (HR_PATH, "HR")]:
    if path and Path(path).exists():
        print(f"✅ {name} dataset trouvé: {path}")
    else:
        print(f"❌ {name} dataset manquant: {path}")

# Vérifier STATIC_PATH, MEAN_PATH, STD_PATH
for path, name in [(STATIC_PATH, "Static"), (MEAN_PATH, "Mean"), (STD_PATH, "Std")]:
    if path and Path(path).exists():
        print(f"✅ {name} trouvé: {path}")
    else:
        print(f"❌ {name} manquant: {path}")
if not Path(STATIC_PATH).exists():
    STATIC_PATH = None
if not Path(MEAN_PATH).exists():
    MEAN_PATH = None
if not Path(STD_PATH).exists():
    STD_PATH = None

# Afficher la structure data/raw/
data_dir = Path("data/raw")
if data_dir.exists():
    print(f"\n📁 Structure data/raw/:")
    for sub in sorted(data_dir.iterdir()):
        if sub.is_dir():
            files = list(sub.glob("*.nc")) or list(sub.glob("*.json")) or []
            print(f"   {sub.name}/: {[f.name for f in files[:5]]}{'...' if len(files) > 5 else ''}")
        else:
            print(f"   {sub.name}")
else:
    print(f"\n⚠️  Répertoire data/raw/ n'existe pas")


### 2.2 Chargement et inspection des données brutes


In [ ]:
# Charger les datasets bruts
lr_raw = xr.open_dataset(LR_PATH)
hr_raw = xr.open_dataset(HR_PATH)

print("=" * 60)
print("📊 LOW RESOLUTION DATASET")
print("=" * 60)
print(lr_raw)
print("\n" + "=" * 60)
print("📊 HIGH RESOLUTION DATASET")
print("=" * 60)
print(hr_raw)


In [ ]:
# Informations sur les dimensions
def print_dataset_info(ds, name):
    print(f"\n{'='*50}")
    print(f"📐 {name} DATASET INFO")
    print(f"{'='*50}")
    print(f"Variables: {list(ds.data_vars)}")
    print(f"Dimensions: {dict(ds.dims)}")
    print(f"Coordinates: {list(ds.coords)}")
    if 'time' in ds.dims or 'time' in ds.coords:
        time_coord = 'time' if 'time' in ds.coords else list(ds.coords)[0]
        try:
            # Gérer différents types de coordonnées temporelles
            time_values = ds[time_coord].values
            if hasattr(time_values[0], 'strftime'):  # datetime object
                start_time = time_values[0]
                end_time = time_values[-1]
            else:  # string ou autre format
                start_time = pd.to_datetime(str(time_values[0]))
                end_time = pd.to_datetime(str(time_values[-1]))
            print(f"Période: {start_time} → {end_time}")
            print(f"Nombre de pas de temps: {len(ds[time_coord])}")
        except Exception as e:
            # En cas d'erreur, afficher les valeurs brutes
            print(f"Coordonnées temporelles: {ds[time_coord].values[0]} → {ds[time_coord].values[-1]}")
            print(f"Nombre de pas de temps: {len(ds[time_coord])}")

print_dataset_info(lr_raw, "LOW RESOLUTION")
print_dataset_info(hr_raw, "HIGH RESOLUTION")


### 2.3 Visualisation des données


In [ ]:
# Visualiser un snapshot des données LR et HR
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# LR data
lr_var = list(lr_raw.data_vars)[0]
lr_raw[lr_var].isel(time=0).plot(ax=axes[0], cmap='viridis')
axes[0].set_title(f'Low Resolution - {lr_var} (t=0)', fontsize=14, fontweight='bold')

# HR data
hr_var = list(hr_raw.data_vars)[0]
hr_raw[hr_var].isel(time=0).plot(ax=axes[1], cmap='RdBu_r')
axes[1].set_title(f'High Resolution - {hr_var} (t=0)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()


### 2.4 Création du pipeline de données


In [ ]:
# Configuration du pipeline (alignée sur downscaling: 8 canaux LR, pr en HR)
SEQ_LEN = 6  # Longueur des séquences temporelles
BASELINE_STRATEGY = "hr_smoothing"  # ou "lr_interp"
BASELINE_FACTOR = 2  # plus petit => plus léger
NORMALIZE = True  # Normalisation LR avec mean/std (comme downscaling)
PRECIPITATION_DELTA = 0.01  # Delta pour log1p (config downscaling: delta=0.01)
NAN_FILL_STRATEGY = "zero"  # Stratégie de nettoyage NaN: "zero", "mean", ou "interpolate"

# Variables comme downscaling (var_names + cible pr)
LR_VARIABLES = ["q_500", "q_850", "u_500", "u_850", "v_500", "v_850", "t_500", "t_850"]  # 8 canaux LR
HR_VARIABLES = ["pr"]  # Cible précipitations (comme train_y dans downscaling)
STATIC_VARIABLES = ["orog", "he", "vegt"] if STATIC_PATH else None  # Champs statiques downscaling

# Sécurité: afficher les chemins réellement utilisés
print(f"LR_PATH actif: {LR_PATH}")
print(f"HR_PATH actif: {HR_PATH}")
print(f"STATIC_PATH: {STATIC_PATH}")
print(f"MEAN_PATH: {MEAN_PATH}, STD_PATH: {STD_PATH}")

# Valider que les variables existent, sinon fallback automatique
import xarray as xr
lr_ds_preview = xr.open_dataset(LR_PATH)
hr_ds_preview = xr.open_dataset(HR_PATH)

lr_avail = set(lr_ds_preview.data_vars)
hr_avail = set(hr_ds_preview.data_vars)

if not set(LR_VARIABLES).issubset(lr_avail):
    print(f"⚠️  LR_VARIABLES invalides: {set(LR_VARIABLES) - lr_avail}")
    LR_VARIABLES = [v for v in LR_VARIABLES if v in lr_avail] or sorted(lr_avail)[:8]
    print(f"➡️  Fallback LR_VARIABLES = {LR_VARIABLES}")

if not set(HR_VARIABLES).issubset(hr_avail):
    print(f"⚠️  HR_VARIABLES invalides: {set(HR_VARIABLES) - hr_avail}")
    HR_VARIABLES = [sorted(hr_avail)[0]]
    print(f"➡️  Fallback HR_VARIABLES = {HR_VARIABLES}")

print("🔄 Création du pipeline de données...")
pipeline = NetCDFDataPipeline(
    lr_path=LR_PATH,
    hr_path=HR_PATH,
    static_path=STATIC_PATH,
    seq_len=SEQ_LEN,
    baseline_strategy=BASELINE_STRATEGY,
    baseline_factor=BASELINE_FACTOR,
    normalize=NORMALIZE,
    nan_fill_strategy=NAN_FILL_STRATEGY,
    precipitation_delta=PRECIPITATION_DELTA,
    lr_variables=LR_VARIABLES,
    hr_variables=HR_VARIABLES,
    static_variables=STATIC_VARIABLES,
    means_path=MEAN_PATH if (MEAN_PATH and os.path.exists(MEAN_PATH)) else None,
    stds_path=STD_PATH if (STD_PATH and os.path.exists(STD_PATH)) else None,
)

print("\n✅ Pipeline créé avec succès!")
print(f"   - LR shape: {pipeline.lr_dataset.dims}")
print(f"   - HR shape: {pipeline.hr_dataset.dims}")
print(f"   - Residual shape: {pipeline.residual_dataset.dims}")


### 2.6 Config test (jeux tenus à l'écart)

Configuration des chemins et GCMs pour l'évaluation sur données de test (comme downscaling : un jeu par GCM dans `data/raw/test/`).

In [ ]:
# Config test : chemins et GCMs (comme downscaling eval_predictor_variables + ground_truth)
TEST_ROOT = Path("data/raw/test")

# Détection des GCMs de test à partir des fichiers présents (convention: {GCM}_histupdated_compressed.nc, {GCM}_historical_precip_compressed.nc)
def _discover_test_gcms():
    if not TEST_ROOT.exists():
        return []
    lr_suffix = "_histupdated_compressed.nc"
    hr_suffix = "_historical_precip_compressed.nc"
    lr_stems = {f.name.replace(lr_suffix, "") for f in TEST_ROOT.glob(f"*{lr_suffix}")}
    hr_stems = {f.name.replace(hr_suffix, "") for f in TEST_ROOT.glob(f"*{hr_suffix}")}
    return sorted(lr_stems & hr_stems)

TEST_GCMs = _discover_test_gcms()
if not TEST_GCMs:
    TEST_GCMs = ["EC-Earth3", "NorESM2-MM"]  # fallback si dossier vide ou absent

def test_lr_path(gcm):
    return str(TEST_ROOT / f"{gcm}_histupdated_compressed.nc")

def test_hr_path(gcm):
    return str(TEST_ROOT / f"{gcm}_historical_precip_compressed.nc")

# Vérifier existence des fichiers pour chaque GCM
print("GCMs de test configurés:", TEST_GCMs)
for gcm in TEST_GCMs:
    lr_ok = Path(test_lr_path(gcm)).exists()
    hr_ok = Path(test_hr_path(gcm)).exists()
    if lr_ok and hr_ok:
        print(f"  OK {gcm}: LR et HR trouvés")
    else:
        print(f"  Manquant {gcm}: LR={lr_ok}, HR={hr_ok}")

In [ ]:
def build_test_pipeline(gcm):
    """
    Construit un pipeline de test pour un GCM (même options que le pipeline d'entraînement).
    Retourne NetCDFDataPipeline ou None si les fichiers sont manquants.
    """
    lr_path = test_lr_path(gcm)
    hr_path = test_hr_path(gcm)
    if not Path(lr_path).exists() or not Path(hr_path).exists():
        print(f"  Fichiers test manquants pour {gcm}, skip.")
        return None
    try:
        return NetCDFDataPipeline(
            lr_path=lr_path,
            hr_path=hr_path,
            static_path=STATIC_PATH,
            seq_len=SEQ_LEN,
            baseline_strategy=BASELINE_STRATEGY,
            baseline_factor=BASELINE_FACTOR,
            normalize=NORMALIZE,
            nan_fill_strategy=NAN_FILL_STRATEGY,
            precipitation_delta=PRECIPITATION_DELTA,
            lr_variables=LR_VARIABLES,
            hr_variables=HR_VARIABLES,
            static_variables=STATIC_VARIABLES,
            means_path=MEAN_PATH if (MEAN_PATH and os.path.exists(MEAN_PATH)) else None,
            stds_path=STD_PATH if (STD_PATH and os.path.exists(STD_PATH)) else None,
        )
    except Exception as e:
        print(f"  Erreur build_test_pipeline({gcm}): {e}")
        return None

In [ ]:
# Visualiser baseline vs HR vs residual
fig, axes = plt.subplots(1, 3, figsize=(20, 5))

time_idx = 0
hr_var = list(pipeline.hr_dataset.data_vars)[0]

# HR truth
pipeline.hr_dataset[hr_var].isel(time=time_idx).plot(ax=axes[0], cmap='RdBu_r')
axes[0].set_title('HR Ground Truth', fontsize=12, fontweight='bold')

# Baseline
pipeline.baseline_prepared[hr_var].isel(time=time_idx).plot(ax=axes[1], cmap='RdBu_r')
axes[1].set_title('Baseline (Deterministic)', fontsize=12, fontweight='bold')

# Residual
pipeline.residual_dataset[hr_var].isel(time=time_idx).plot(ax=axes[2], cmap='seismic', center=0)
axes[2].set_title('Residual (HR - Baseline)', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()


### 2.5 Création du dataset iterable


In [ ]:
# Créer le dataset PyTorch
dataset = pipeline.build_sequence_dataset(
    seq_len=SEQ_LEN,
    stride=1,
    as_torch=True,
)

# Inspecter un batch
sample = next(iter(dataset))

print("📦 Structure d'un batch:")
print("=" * 60)
for key, value in sample.items():
    if torch.is_tensor(value):
        print(f"  {key:15s}: shape={tuple(value.shape)}, dtype={value.dtype}")
    else:
        print(f"  {key:15s}: {type(value).__name__}")

print(f"\n✅ Dataset créé avec succès!")
print(f"   Longueur de séquence: {SEQ_LEN}")


## 🏗️ 3. Construction du Modèle

### 3.1 Configuration du modèle


In [ ]:
# Configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Dimensions (à adapter selon vos données)
lr_shape = tuple(sample["lr"].shape[2:4])  # Extraire (lat, lon) de LR
hr_shape = tuple(sample["residual"].shape[2:4])  # Extraire (lat, lon) de HR

# Architecture (réduite pour overfit rapide)
HIDDEN_DIM = 32
CONDITIONING_DIM = 32
RCN_DRIVER_DIM = sample["lr"].shape[1]  # Nombre de channels LR
RCN_RECONSTRUCTION_DIM = sample["lr"].shape[1]
DIFFUSION_STEPS = 50

# Hyperparamètres d'entraînement
LEARNING_RATE = 5e-5  # Réduit de 3e-4 à 5e-5 (6x plus bas) pour éviter la divergence
NUM_EPOCHS = 6
GRADIENT_CLIPPING = 0.5  # Réduit de 1.0 à 0.5 pour stabiliser l'entraînement

# Pondération des pertes (simplifiée pour test "mémorisation")
LAMBDA_GEN = 1.0   # Perte de diffusion
BETA_REC = 0.0     # désactivée pour simplifier
GAMMA_DAG = 0.0    # désactivée pour simplifier

print(f"🖥️  Device: {DEVICE}")
print(f"📐 LR shape: {lr_shape}")
print(f"📐 HR shape: {hr_shape}")
print(f"📐 LR channels: {RCN_DRIVER_DIM}")
print(f"📐 HR channels: {sample['residual'].shape[1]}")


### 3.2 Construction du graph builder


In [ ]:
# Graph builder pour les relations spatiales
builder = HeteroGraphBuilder(
    lr_shape=lr_shape,
    hr_shape=hr_shape,
    static_dataset=pipeline.get_static_dataset(),
    include_mid_layer=False,
)

print("✅ Graph builder créé")
print(f"   Dynamic node types: {builder.dynamic_node_types}")
print(f"   Static node types: {builder.static_node_types}")


### 3.3 Initialisation des modules du modèle


In [ ]:
# 1. Intelligible Variable Encoder
encoder_configs = [
    IntelligibleVariableConfig(
        name="surface",
        meta_path=("GP850", "spat_adj", "GP850"),
        pool="mean"
    ),
]

if pipeline.get_static_dataset() is not None:
    encoder_configs.append(
        IntelligibleVariableConfig(
            name="static",
            meta_path=("SP_HR", "causes", "GP850"),
            pool="mean"
        )
    )

encoder = IntelligibleVariableEncoder(
    configs=encoder_configs,
    hidden_dim=HIDDEN_DIM,
    conditioning_dim=CONDITIONING_DIM,
).to(DEVICE)

num_vars = len(encoder_configs)
print(f"✅ Encoder créé avec {num_vars} variables intelligibles")

# 2. Causal RCN
rcn_cell = RCNCell(
    num_vars=num_vars,
    hidden_dim=HIDDEN_DIM,
    driver_dim=RCN_DRIVER_DIM,
    reconstruction_dim=RCN_RECONSTRUCTION_DIM,
    dropout=0.0,
).to(DEVICE)

rcn_runner = RCNSequenceRunner(rcn_cell, detach_interval=None)
print(f"✅ RCN créé")

# 3. Diffusion Decoder (UNet réduit pour test rapide)
hr_channels = sample['residual'].shape[1]

# UNet minimal pour accepter des tailles non multiples de 2^n (ex: 23x26)
UNET_KWARGS = dict(
    layers_per_block=1,
    block_out_channels=(32,),
    down_block_types=("DownBlock2D",),
    up_block_types=("UpBlock2D",),
    norm_num_groups=8,
)

diffusion = CausalDiffusionDecoder(
    in_channels=hr_channels,
    conditioning_dim=CONDITIONING_DIM,
    height=hr_shape[0],
    width=hr_shape[1],
    num_diffusion_steps=DIFFUSION_STEPS,
    unet_kwargs=UNET_KWARGS,
).to(DEVICE)

print(f"✅ Diffusion decoder créé avec {DIFFUSION_STEPS} steps")

# 4. Optimizer
params = list(encoder.parameters()) + list(rcn_cell.parameters()) + list(diffusion.parameters())
optimizer = torch.optim.Adam(params, lr=LEARNING_RATE)

print(f"✅ Optimizer créé (lr={LEARNING_RATE})")

# Compter les paramètres (en gérant les paramètres non initialisés/lazy)
def count_parameters(model):
    """Compte les paramètres d'un modèle, en gérant les paramètres non initialisés."""
    total = 0
    trainable = 0
    for p in model.parameters():
        try:
            num = p.numel()
            total += num
            if p.requires_grad:
                trainable += num
        except (ValueError, RuntimeError):
            # Paramètre non initialisé (lazy module), on l'ignore pour le moment
            pass
    return total, trainable

total_params = 0
trainable_params = 0
for model in [encoder, rcn_cell, diffusion]:
    total, trainable = count_parameters(model)
    total_params += total
    trainable_params += trainable

print(f"\n📊 Nombre de paramètres:")
print(f"   Total: {total_params:,}")
print(f"   Trainable: {trainable_params:,}")


## 🚀 4. Entraînement du Modèle

### 4.1 Fonction helper pour la conversion des batches


In [ ]:
def convert_sample_to_batch(sample, builder, device):
    """
    Convertit un échantillon du DataLoader en format attendu par train_epoch.
    """
    lr_seq = sample["lr"]  # [seq_len, channels, lat, lon]
    seq_len = lr_seq.shape[0]
    
    # Convertir grille LR en nodes
    lr_nodes_steps = []
    for t in range(seq_len):
        lr_nodes_steps.append(builder.lr_grid_to_nodes(lr_seq[t]))
    lr_tensor = torch.stack(lr_nodes_steps, dim=0)  # [seq_len, N_lr, channels]
    
    # Préparer hetero data
    dynamic_features = {node_type: lr_nodes_steps[0] for node_type in builder.dynamic_node_types}
    hetero = builder.prepare_step_data(dynamic_features).to(device)
    
    batch = {
        "lr": lr_tensor,
        "residual": sample["residual"],
        "baseline": sample.get("baseline"),
        "hetero": hetero,
    }
    return batch

def iterate_batches(dataloader, builder, device):
    """Générateur qui convertit chaque sample du dataloader."""
    for sample in dataloader:
        yield convert_sample_to_batch(sample, builder, device)


### 4.2 Boucle d'entraînement


In [ ]:
# Mode overfit: on fixe UN batch et on le répète pour vérifier que le modèle mémorise
fixed_sample = sample
fixed_batch = convert_sample_to_batch(fixed_sample, builder, DEVICE)

def repeat_fixed_batch(n_steps: int):
    for _ in range(n_steps):
        batch = {}
        for k, v in fixed_batch.items():
            batch[k] = v.to(DEVICE) if torch.is_tensor(v) else v
        yield batch

history = {k: [] for k in ["loss", "loss_gen", "loss_rec", "loss_dag"]}

STEPS_PER_EPOCH = 10

print("🏋️ Début overfit (mémorisation)...")
print("=" * 80)

for epoch in range(NUM_EPOCHS):
    metrics = train_epoch(
        encoder=encoder,
        rcn_runner=rcn_runner,
        diffusion_decoder=diffusion,
        optimizer=optimizer,
        data_loader=repeat_fixed_batch(STEPS_PER_EPOCH),
        lambda_gen=LAMBDA_GEN,
        beta_rec=BETA_REC,
        gamma_dag=GAMMA_DAG,
        conditioning_fn=None,
        device=DEVICE,
        gradient_clipping=GRADIENT_CLIPPING,
    )

    for key in history.keys():
        history[key].append(metrics[key])

    print(
        f"Epoch {epoch + 1}/{NUM_EPOCHS} | "
        f"Loss: {metrics['loss']:.4f} | "
        f"Gen: {metrics['loss_gen']:.4f} | "
        f"Rec: {metrics['loss_rec']:.4f} | "
        f"DAG: {metrics['loss_dag']:.4f}"
    )

print("\n" + "=" * 80)
print("✅ Overfit terminé!")


### 4.3 Visualisation de l'entraînement


In [ ]:
# Courbes de loss
if len(history["loss"]) == 0:
    raise RuntimeError("Aucune métrique dans history: entraînement non exécuté.")

fig, axes = plt.subplots(2, 2, figsize=(16, 10))

epochs = range(1, len(history["loss"]) + 1)

# Loss totale
axes[0, 0].plot(epochs, history['loss'], linewidth=2, color='navy')
axes[0, 0].set_title('Total Loss', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].grid(True, alpha=0.3)

# Loss de génération (diffusion)
axes[0, 1].plot(epochs, history['loss_gen'], linewidth=2, color='crimson')
axes[0, 1].set_title('Generation Loss (Diffusion)', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].grid(True, alpha=0.3)

# Loss de reconstruction
axes[1, 0].plot(epochs, history['loss_rec'], linewidth=2, color='green')
axes[1, 0].set_title('Reconstruction Loss', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Loss')
axes[1, 0].grid(True, alpha=0.3)

# Contrainte DAG (NO TEARS)
axes[1, 1].plot(epochs, history['loss_dag'], linewidth=2, color='orange')
axes[1, 1].set_title('DAG Constraint (NO TEARS)', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Loss')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].axhline(y=0, color='red', linestyle='--', alpha=0.5, label='Target')
axes[1, 1].legend()

plt.tight_layout()
plt.savefig('training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Courbes sauvegardées dans 'training_curves.png'")


### 4.4 Sauvegarde du modèle


In [ ]:
# Sauvegarder le checkpoint
checkpoint_path = "st_cdgm_checkpoint.pth"

torch.save({
    'epoch': NUM_EPOCHS,
    'encoder_state_dict': encoder.state_dict(),
    'rcn_cell_state_dict': rcn_cell.state_dict(),
    'diffusion_state_dict': diffusion.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'history': history,
    'config': {
        'hidden_dim': HIDDEN_DIM,
        'conditioning_dim': CONDITIONING_DIM,
        'diffusion_steps': DIFFUSION_STEPS,
        'lr_shape': lr_shape,
        'hr_shape': hr_shape,
    }
}, checkpoint_path)

print(f"✅ Checkpoint sauvegardé: {checkpoint_path}")
print(f"   Taille: {Path(checkpoint_path).stat().st_size / 1024 / 1024:.2f} MB")


## 🧪 5. Évaluation et Tests

### 5.1 Fonction d'inférence


In [ ]:
@torch.no_grad()
def generate_prediction(encoder, rcn_runner, diffusion, sample, builder, device):
    """Génère une prédiction haute résolution à partir d'un échantillon.

    Note (mode overfit): ici on prédit 1 canal (ex: t_850). La branche
    "contraintes physiques" du decoder attend 3 canaux (T_min, Δ1, Δ2), donc
    on désactive `apply_constraints` pendant le sampling.
    """
    encoder.eval()
    rcn_runner.cell.eval()
    diffusion.eval()

    # Convertir le sample
    batch = convert_sample_to_batch(sample, builder, device)

    lr_data = batch["lr"].to(device)
    hetero_data = batch["hetero"]

    # Forward pass
    H_init = encoder.init_state(hetero_data).to(device)
    drivers = [lr_data[t] for t in range(lr_data.shape[0])]
    seq_output = rcn_runner.run(H_init, drivers, reconstruction_sources=None)

    # Conditioning pour la diffusion
    H_condition = seq_output.states[-1]
    conditioning = encoder.project_state_tensor(H_condition).to(device)

    # Génération par diffusion (sampling)
    # - apply_constraints=False: compatible sorties 1 canal
    generated = diffusion.sample(
        conditioning,
        num_steps=25,
        scheduler_type="ddpm",
        apply_constraints=False,
    )

    return generated.residual, batch["residual"][-1].to(device), batch["baseline"][-1].to(device)

print("✅ Fonction d'inférence définie")


### 5.2 Génération de prédictions


In [ ]:
# Prendre quelques échantillons de test
test_dataset = pipeline.build_sequence_dataset(seq_len=SEQ_LEN, as_torch=True)
test_samples = [next(iter(test_dataset)) for _ in range(3)]  # 3 échantillons

print(f"📊 {len(test_samples)} échantillons de test préparés")


In [ ]:
# Générer des prédictions
predictions = []
targets = []
baselines = []

print("🔮 Génération des prédictions...")
for i, sample in enumerate(test_samples):
    pred, target, baseline = generate_prediction(encoder, rcn_runner, diffusion, sample, builder, DEVICE)
    predictions.append(pred.cpu())
    targets.append(target.cpu())
    baselines.append(baseline.cpu())
    print(f"  Sample {i+1}/{len(test_samples)} traité")

print("✅ Prédictions générées")


In [ ]:
# Diagnostic des données avant calcul des métriques
print("\n🔍 Diagnostic des données:")
print("=" * 80)

for i, (pred, target, baseline) in enumerate(zip(predictions, targets, baselines)):
    pred_squeezed = pred.squeeze(0) if pred.dim() == 4 else pred
    full_pred = baselines[i] + pred_squeezed
    full_target = baselines[i] + target
    
    # Vérifications
    pred_nan_count = torch.isnan(full_pred).sum().item()
    target_nan_count = torch.isnan(full_target).sum().item()
    pred_inf_count = torch.isinf(full_pred).sum().item()
    target_inf_count = torch.isinf(full_target).sum().item()
    
    print(f"\nSample {i+1}:")
    print(f"  Prédictions - NaN: {pred_nan_count}, Inf: {pred_inf_count}")
    print(f"  Cibles - NaN: {target_nan_count}, Inf: {target_inf_count}")
    
    if pred_nan_count > 0 or target_nan_count > 0:
        print(f"  ⚠️  NaN détectés - Nettoyage automatique...")
        full_pred = torch.nan_to_num(full_pred, nan=0.0, posinf=0.0, neginf=0.0)
        full_target = torch.nan_to_num(full_target, nan=0.0, posinf=0.0, neginf=0.0)
        predictions[i] = full_pred - baselines[i]
        targets[i] = full_target - baselines[i]
    
    # Statistiques sur les valeurs valides
    valid_mask = ~torch.isnan(full_target) & ~torch.isnan(full_pred)
    if valid_mask.sum() > 0:
        pred_valid = full_pred[valid_mask]
        target_valid = full_target[valid_mask]
        print(f"  Valeurs valides: {valid_mask.sum().item()}/{valid_mask.numel()}")
        print(f"  Prédiction - Min: {pred_valid.min():.6f}, Max: {pred_valid.max():.6f}, Mean: {pred_valid.mean():.6f}, Std: {pred_valid.std():.6f}")
        print(f"  Cible - Min: {target_valid.min():.6f}, Max: {target_valid.max():.6f}, Mean: {target_valid.mean():.6f}, Std: {target_valid.std():.6f}")
        
        # Avertissement si variance très faible
        if target_valid.std() < 1e-6:
            print(f"  ⚠️  Variance de la cible très faible ({target_valid.std():.6e}) - risque de NaN dans corrélation")
        if pred_valid.std() < 1e-6:
            print(f"  ⚠️  Variance de la prédiction très faible ({pred_valid.std():.6e}) - risque de NaN dans corrélation")

print("\n✅ Diagnostic terminé")

### 5.3 Calcul des métriques


In [ ]:
def compute_metrics(pred, target, mask=None):
    """
    Calcule des métriques robustes (MSE, RMSE, MAE, Corrélation) en gérant les NaNs et la stabilité.
    
    Args:
        pred (torch.Tensor): Sorties du modèle.
        target (torch.Tensor): Données de vérité terrain (Ground Truth).
        mask (torch.Tensor, optional): Masque booléen où 1 est valide, 0 est invalide/remplissage.
        
    Returns:
        dict: Dictionnaire contenant les valeurs scalaires des métriques.
    """
    # 1. Créer un masque de validité
    if mask is None:
        mask = ~torch.isnan(target) & ~torch.isnan(pred) & ~torch.isinf(target) & ~torch.isinf(pred)
    
    # 2. Aplatir et Filtrer
    pred_flat = pred.flatten()
    target_flat = target.flatten()
    
    if mask.dim() > 0:
        mask_flat = mask.flatten()
    else:
        mask_flat = mask
    
    pred_valid = pred_flat[mask_flat]
    target_valid = target_flat[mask_flat]
    
    # Vérifier si nous avons des données valides
    if pred_valid.numel() == 0:
        print("[WARN] Aucune donnée valide pour le calcul des métriques")
        return {
            "MSE": 0.0, "RMSE": 0.0, "MAE": 0.0, "Correlation": 0.0
        }

    # 3. Calculer les Métriques de Distance
    mse = torch.mean((pred_valid - target_valid) ** 2)
    rmse = torch.sqrt(mse)
    mae = torch.mean(torch.abs(pred_valid - target_valid))

    # 4. Calculer la Corrélation de Pearson avec Stabilité
    vx = pred_valid - torch.mean(pred_valid)
    vy = target_valid - torch.mean(target_valid)
    
    epsilon = 1e-8
    numerator = torch.sum(vx * vy)
    denominator = torch.sqrt(torch.sum(vx ** 2)) * torch.sqrt(torch.sum(vy ** 2))
    
    if denominator < epsilon:
        correlation = torch.tensor(0.0)
    else:
        correlation = numerator / (denominator + epsilon)

    return {
        "MSE": mse.item(),
        "RMSE": rmse.item(),
        "MAE": mae.item(),
        "Correlation": correlation.item()
    }

# Calculer les métriques pour chaque échantillon
print("\n📈 Métriques de performance:")
print("=" * 80)

all_metrics = []
for i, (pred, target) in enumerate(zip(predictions, targets)):
    # Reconstruction complète: baseline + residual prédit
    # pred peut avoir une dimension batch [1, C, H, W], on la supprime avec squeeze
    pred_squeezed = pred.squeeze(0) if pred.dim() == 4 else pred
    full_pred = baselines[i] + pred_squeezed
    full_target = baselines[i] + target
    
    metrics = compute_metrics(full_pred, full_target)
    all_metrics.append(metrics)
    
    print(f"\nSample {i+1}:")
    for metric_name, value in metrics.items():
        print(f"  {metric_name:15s}: {value:.6f}")

# Moyennes
print("\n" + "=" * 80)
print("📊 Moyennes sur tous les échantillons:")
avg_metrics = {key: np.mean([m[key] for m in all_metrics]) for key in all_metrics[0].keys()}
for metric_name, value in avg_metrics.items():
    print(f"  {metric_name:15s}: {value:.6f}")


### 5.4 Évaluation sur jeux de test par GCM

Évaluation sur données tenues à l'écart (data/raw/test/), un pipeline par GCM, même normalisation que l'entraînement. Inférence puis métriques par GCM.

In [ ]:
# Mode évaluation et nombre d'échantillons de test par GCM
encoder.eval()
rcn_runner.cell.eval()
diffusion.eval()
N_TEST_SAMPLES = 10  # nombre d'échantillons à évaluer par GCM (ajuster si besoin)

In [ ]:
# Boucle de test par GCM (données tenues à l'écart)
test_results = {}
for gcm in TEST_GCMs:
    pipe_test = build_test_pipeline(gcm)
    if pipe_test is None:
        continue
    test_dataset = pipe_test.build_sequence_dataset(seq_len=SEQ_LEN, as_torch=True)
    # Prendre au plus N_TEST_SAMPLES échantillons
    test_samples_gcm = []
    it = iter(test_dataset)
    for _ in range(N_TEST_SAMPLES):
        try:
            test_samples_gcm.append(next(it))
        except StopIteration:
            break
    if not test_samples_gcm:
        print(f"  Aucun échantillon pour {gcm}, skip.")
        continue
    predictions_gcm, targets_gcm, baselines_gcm = [], [], []
    for sample in test_samples_gcm:
        pred, target, baseline = generate_prediction(encoder, rcn_runner, diffusion, sample, builder, DEVICE)
        predictions_gcm.append(pred.cpu())
        targets_gcm.append(target.cpu())
        baselines_gcm.append(baseline.cpu())
    # Métriques par échantillon puis moyenne pour ce GCM
    all_metrics_gcm = []
    for i, (pred, target) in enumerate(zip(predictions_gcm, targets_gcm)):
        pred_sq = pred.squeeze(0) if pred.dim() == 4 else pred
        full_pred = baselines_gcm[i] + pred_sq
        full_target = baselines_gcm[i] + target
        full_pred = torch.nan_to_num(full_pred, nan=0.0, posinf=0.0, neginf=0.0)
        full_target = torch.nan_to_num(full_target, nan=0.0, posinf=0.0, neginf=0.0)
        m = compute_metrics(full_pred, full_target)
        all_metrics_gcm.append(m)
    avg_gcm = {k: np.mean([m[k] for m in all_metrics_gcm]) for k in all_metrics_gcm[0].keys()}
    test_results[gcm] = avg_gcm
    print(f"  {gcm}: MSE={avg_gcm['MSE']:.6f}, MAE={avg_gcm['MAE']:.6f}, Corr={avg_gcm['Correlation']:.6f}")
print("Évaluation par GCM terminée.")

In [ ]:
# Affichage des métriques par GCM et export CSV
if test_results:
    metrics_df = pd.DataFrame(test_results).T
    metrics_df.index.name = "GCM"
    print("Métriques par GCM (jeux de test):")
    print(metrics_df.to_string())
    out_csv = "test_metrics_by_gcm.csv"
    metrics_df.to_csv(out_csv)
    print(f"\nMétriques exportées dans '{out_csv}'")
else:
    print("Aucun résultat de test (test_results vide). Vérifier TEST_GCMs et les fichiers dans data/raw/test/.")

## 📊 6. Visualisation des Résultats

### 6.1 Comparaison visuelle: Baseline vs Prédiction vs Ground Truth


In [ ]:
# Visualiser les résultats pour le premier échantillon
# Import nécessaire pour CenteredNorm (centrage de colormap)
import matplotlib.colors as mcolors

sample_idx = 0

# predictions[sample_idx] est maintenant directement un tenseur [batch, C, H, W] ou [C, H, W]
# On squeeze pour enlever la dimension batch si elle existe
pred_residual = predictions[sample_idx].squeeze().numpy()  # [C, H, W]
target_residual = targets[sample_idx].squeeze().numpy()
baseline = baselines[sample_idx].squeeze().numpy()

# Reconstruction complète
full_pred = baseline + pred_residual
full_target = baseline + target_residual

# Sélectionner le premier canal si multi-canal
if len(full_pred.shape) > 2:
    channel_idx = 0
    pred_plot = full_pred[channel_idx]
    target_plot = full_target[channel_idx]
    baseline_plot = baseline[channel_idx]
else:
    pred_plot = full_pred
    target_plot = full_target
    baseline_plot = baseline

# Créer la visualisation
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Trouver les valeurs min/max pour une échelle commune
vmin = min(baseline_plot.min(), pred_plot.min(), target_plot.min())
vmax = max(baseline_plot.max(), pred_plot.max(), target_plot.max())

# Première ligne: Baseline, Prédiction, Ground Truth
im1 = axes[0, 0].imshow(baseline_plot, cmap='RdBu_r', vmin=vmin, vmax=vmax)
axes[0, 0].set_title('Baseline (Deterministic)', fontsize=14, fontweight='bold')
axes[0, 0].axis('off')
plt.colorbar(im1, ax=axes[0, 0], fraction=0.046)

im2 = axes[0, 1].imshow(pred_plot, cmap='RdBu_r', vmin=vmin, vmax=vmax)
axes[0, 1].set_title('Prediction (Baseline + Diffusion)', fontsize=14, fontweight='bold')
axes[0, 1].axis('off')
plt.colorbar(im2, ax=axes[0, 1], fraction=0.046)

im3 = axes[0, 2].imshow(target_plot, cmap='RdBu_r', vmin=vmin, vmax=vmax)
axes[0, 2].set_title('Ground Truth (HR)', fontsize=14, fontweight='bold')
axes[0, 2].axis('off')
plt.colorbar(im3, ax=axes[0, 2], fraction=0.046)

# Deuxième ligne: Erreurs
baseline_error = np.abs(baseline_plot - target_plot)
pred_error = np.abs(pred_plot - target_plot)
error_diff = baseline_error - pred_error  # Positive = amélioration

im4 = axes[1, 0].imshow(baseline_error, cmap='Reds')
axes[1, 0].set_title(f'Baseline Error (MAE={baseline_error.mean():.4f})', fontsize=12, fontweight='bold')
axes[1, 0].axis('off')
plt.colorbar(im4, ax=axes[1, 0], fraction=0.046)

im5 = axes[1, 1].imshow(pred_error, cmap='Reds')
axes[1, 1].set_title(f'Prediction Error (MAE={pred_error.mean():.4f})', fontsize=12, fontweight='bold')
axes[1, 1].axis('off')
plt.colorbar(im5, ax=axes[1, 1], fraction=0.046)

# CORRECTION: Utiliser CenteredNorm pour centrer la colormap 'seismic' sur 0
# matplotlib.imshow() n'accepte pas l'argument 'center' (c'est une fonctionnalité de seaborn.heatmap)
im6 = axes[1, 2].imshow(error_diff, cmap='seismic', norm=mcolors.CenteredNorm(vcenter=0))
axes[1, 2].set_title('Improvement (Baseline - Prediction)', fontsize=12, fontweight='bold')
axes[1, 2].axis('off')
plt.colorbar(im6, ax=axes[1, 2], fraction=0.046)

plt.tight_layout()
plt.savefig('prediction_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Visualisation sauvegardée dans 'prediction_comparison.png'")


In [ ]:
# Sauvegarder les métriques dans un fichier CSV
metrics_df = pd.DataFrame(all_metrics)
metrics_df['sample'] = range(1, len(all_metrics) + 1)
metrics_df = metrics_df[['sample'] + [col for col in metrics_df.columns if col != 'sample']]

# Ajouter la ligne des moyennes
avg_row = pd.DataFrame([avg_metrics])
avg_row.insert(0, 'sample', 'Average')
metrics_df = pd.concat([metrics_df, avg_row], ignore_index=True)

metrics_df.to_csv('evaluation_metrics.csv', index=False)
print("✅ Métriques sauvegardées dans 'evaluation_metrics.csv'")
print("\n", metrics_df.to_string(index=False))


## 🎯 7. Résumé et Conclusions


In [ ]:
print("\n" + "=" * 80)
print("🎉 RÉSUMÉ DE L'EXPÉRIENCE ST-CDGM")
print("=" * 80)
print(f"\n📁 Configuration:")
print(f"   - LR dataset: {LR_PATH}")
print(f"   - HR dataset: {HR_PATH}")
print(f"   - Sequence length: {SEQ_LEN}")
print(f"   - LR shape: {lr_shape}")
print(f"   - HR shape: {hr_shape}")

print(f"\n🏗️  Architecture:")
print(f"   - Hidden dimension: {HIDDEN_DIM}")
print(f"   - Conditioning dimension: {CONDITIONING_DIM}")
print(f"   - Diffusion steps: {DIFFUSION_STEPS}")
print(f"   - Total parameters: {total_params:,}")

print(f"\n🚀 Entraînement:")
print(f"   - Epochs: {NUM_EPOCHS}")
print(f"   - Learning rate: {LEARNING_RATE}")
print(f"   - Final total loss: {history['loss'][-1]:.6f}")
print(f"   - Final gen loss: {history['loss_gen'][-1]:.6f}")
print(f"   - Final rec loss: {history['loss_rec'][-1]:.6f}")
print(f"   - Final DAG loss: {history['loss_dag'][-1]:.6f}")

print(f"\n📊 Performance (moyennes):")
for metric_name, value in avg_metrics.items():
    print(f"   - {metric_name}: {value:.6f}")

print("\n" + "=" * 80)
print("✅ Notebook exécuté avec succès!")
print("=" * 80)


---

## 📚 Notes et Prochaines Étapes

### Interprétation des résultats :

1. **Loss totale** : Devrait diminuer au fil des epochs
2. **Loss de génération (diffusion)** : Mesure la qualité du modèle de diffusion
3. **Loss de reconstruction** : Mesure la capacité à reconstruire les inputs LR
4. **DAG constraint (NO TEARS)** : Devrait tendre vers 0 pour assurer l'acyclicité du graphe causal

### Améliorations possibles :

- Augmenter le nombre d'epochs (actuellement 20)
- Ajuster les hyperparamètres (learning rate, hidden dim, etc.)
- Essayer différentes stratégies de baseline
- Ajouter des transformations de données (log, normalisation)
- Implémenter un scheduler de learning rate
- Ajouter de la validation pendant l'entraînement

### Pour aller plus loin :

- Analyser les graphes causaux appris (matrices A)
- Comparer avec d'autres méthodes de downscaling
- Évaluer sur différentes régions géographiques
- Tester la robustesse aux conditions extrêmes
